<a href="https://colab.research.google.com/github/vkayonline/US-COVID19-Forecasting/blob/main/TSF_CT2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

1. Read the data and do exploratory data analysis. Describe the data briefly and interpret the inferences for each. Initial steps like head() .info(), Data Types, etc .

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/vkayonline/US-COVID19-Forecasting/main/datasets/us_covid19_daily.csv')
data.head()

,Date,Positive,Hospitalized,Death
0,17-03-2020,10021,325,124
1,18-03-2020,13385,416,155
2,19-03-2020,18085,617,203
3,20-03-2020,24197,1042,273
4,21-03-2020,31013,1492,335


In [3]:
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')

In [4]:
data.head()

,Positive,Hospitalized,Death
Date,,,
2020-03-17,10021,325,124
2020-03-18,13385,416,155
2020-03-19,18085,617,203
2020-03-20,24197,1042,273
2020-03-21,31013,1492,335


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          265 non-null    object
 1   Positive      265 non-null    int64 
 2   Hospitalized  265 non-null    int64 
 3   Death         265 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 8.4+ KB


2. Check for null values. Display appropriate plots for the columns and discuss inference from the plots and summary stats. Check for null values.

3. Display boxplot to find the distribution of the variable ‘hospitalized’ and interpret your inferences. Perform boxplot for month wise distribution of the same and explain your inferences.

4. Perform decomposition of the time series data.

5. Is the data stationary? Plot original data and the rolling mean and standard deviation with window = 15. Perform statistical tests to confirm the stationarity of the data.

6. Plot ACF and PACF plots for the series.

7. Declare the feature vector and split the data into training and test sets. Consider the data before 06-11-2020 for training data and the rest for the test set.

8. Perform Simple average model, Exponential Model, AR, MA and ARIMA Model,Compare the Evaluation metrics of the models and interpret your inferences.